In [1]:
from dataclasses import dataclass, asdict
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser

%load_ext sql
%config SqlMagic.displaylimit = 10

Deploy Flask apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [3]:
config = ConfigParser()
config.read("../../.config")

['../../.config']

In [4]:
api_key = config["GoogleMaps"]["API_KEY"]
base_url = config["GoogleMaps"]["base_url"]
radius = 50000  # in meters
place_type = "shopping"  # Example place type
address = "Luxembourg City, Luxembourg"

In [4]:
# Connect to DuckDB
# con = duckdb.connect('places.db')
# df = con.execute('SELECT * FROM places').fetchdf()

In [5]:
conn = duckdb.connect("../../data/raw/database.db")
%sql conn --alias duckdb

In [6]:
%%sql
select column_name from  information_schema.columns
where table_name = 'places';

Running query in 'duckdb'

column_name
place_id
name
vicinity
latitude
longitude


In [8]:
%%sql
select count(1) from geonames

Running query in 'duckdb'

count(1)
1259


In [22]:
%%sql

show tables;

Running query in 'duckdb'

name
geonames
place_details
places


In [14]:
# conn.close()

In [7]:
%%sql

select distinct * from places;

Running query in 'duckdb'

place_id,name,vicinity,latitude,longitude
